<a href="https://www.kaggle.com/code/hydramst/water-pollution-lr?scriptVersionId=98070414" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water-pollution/Submission.csv
/kaggle/input/water-pollution/Target.csv
/kaggle/input/water-pollution/Train.csv
/kaggle/input/water-pollution/Test.csv


In [114]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/Submission.csv', delimiter = ',')

# Check if ID's in train presented in train

In [93]:
print('intersection of train and test ID\'s:', 
      len(set(DF_train['id'].values).intersection(set(DF_test['id'].values)))/len(set(DF_test['id'].values)))

intersection of train and test ID's: 0.6333333333333333


63% of test ID's presented in train

In [115]:
ids_intest = pd.DataFrame(DF_test['id'].values)
ids_intrain = pd.DataFrame(set(DF_train['id'].values).intersection(set(DF_test['id'].values)))

In [116]:
in_train = []
notin_train = []

for value in ids_intest.values:
    if (value in ids_intrain.values):
        in_train +=[value]
    else:
        notin_train +=[value]
        
id_notin_train = pd.DataFrame(notin_train, columns=['id'])
id_in_train = pd.DataFrame(in_train, columns=['id'])

print('ID presented in train:',len(in_train))

print( 'ID not presented in train:', len(notin_train))

ID presented in train: 78
ID not presented in train: 44


### Concatinate train and test

In [117]:
df = pd.concat([DF_train, DF_test])

In [ ]:
df = df.drop(columns = ['Unnamed: 0'])

## View categorical and numerical columns

In [119]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['code', 'period', 'id', 'Country'] 
 len =  4
numerical columns:	  ['year', 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 
 len =  13


## Move 'year' to num columns 

In [120]:
cat_columns = ['code', 'period', 'id', 'Country', 'year']
num_columns = [ 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 


### Fill NaN values with mean and ffill (for object) values

In [121]:
df = pd.concat([df, id_notin_train])

In [122]:
df[num_columns].fillna(df[num_columns].mean())
df = df.ffill()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2484 entries, 0 to 43
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code             2484 non-null   object 
 1   year             2484 non-null   float64
 2   period           2484 non-null   object 
 3   id               2484 non-null   object 
 4   Country          2484 non-null   object 
 5   tourists         2484 non-null   float64
 6   venue            2484 non-null   float64
 7   rate             2484 non-null   float64
 8   food             2484 non-null   float64
 9   glass            2484 non-null   float64
 10  metal            2484 non-null   float64
 11  other            2484 non-null   float64
 12  paper            2484 non-null   float64
 13  plastic          2484 non-null   float64
 14  leather          2484 non-null   float64
 15  green_waste      2484 non-null   float64
 16  waste_recycling  2484 non-null   float64
dtypes: float64(13), 

### Fill missing Target values with mean

In [192]:
mean_targ = Target['polution'].mean()
targ = []
for i in range(0, len(id_notin_train)):
    targ += [mean_targ]
    
Target = pd.concat([Target, pd.DataFrame(targ, columns=['polution'])])

,Unnamed: 0,polution
0,6.0,0.013212
1,10.0,0.057482
2,17.0,0.037835
3,24.0,0.055119
4,34.0,0.037617
...,...,...
39,NaN,0.082853
40,NaN,0.082853
41,NaN,0.082853
42,NaN,0.082853


## OHE 

In [193]:
df_ohe = pd.get_dummies(df[cat_columns])
df_all = pd.concat([df_ohe, df[num_columns]], axis=1)

# Scaling

In [194]:
# Creating StandardScaler Object
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)

## Split back to test and train 


In [195]:
len_test = len(DF_test)
train = df_all.iloc[:-len_test]
test = df_all.iloc[-len_test:]
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

2318 2362 122 122


# Log Scaling of Target

In [203]:
target_log = np.log(Target['polution'])
target_orig = Target['polution']

# Traintest split

In [204]:
X, y = pd.DataFrame(train).values, target_log.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

(1889, 473, 1889, 473)

In [198]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1889, 1562), (473, 1562), (1889,), (473,))

# GridSearchCV

In [205]:
### define model
model = Ridge()

alphas = np.array([1,0.1,0.01,0.001,0.0001,0.000001, 0.0000001])

# define parameters
param = {
    'alpha':alphas,
    'positive':[True, False],
    'copy_X':[True,False],
    'fit_intercept':[True,False]
    
}
#grid = GridSearchCV(model, param, scoring='r2', cv=2, n_jobs=-1)
#grid.fit(X_train, y_train)
#predictions = grid.predict(X_test)
#print('R2 : %.4f' %  r2_score(y_test,predictions))
#print('R2 : %.4f' % grid.best_score_)

# ElasticNetCV

In [ ]:
model_ElasticCV = ElasticNetCV(n_alphas=1, fit_intercept=True, max_iter=100000, l1_ratio=0.9, eps=0.0001, cv=5)

#model_ElasticCV.fit(X_train, y_train)

#y_predict_ElasticCV = model_ElasticCV.predict(X_test)

#print('R2 : %.4f' %  r2_score(y_test,y_predict_ElasticCV))

# ElasticNet

In [ ]:
model_Elastic = ElasticNet(alpha=0.00005, fit_intercept=True, max_iter=100000, l1_ratio=0.07)

model_Elastic.fit(X_train, y_train)

y_predict_Elastic = model_Elastic.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict_Elastic))

# Ridge LR

In [206]:
model = Ridge(alpha = 0.1, fit_intercept=True, max_iter=10000)

model.fit(X_train, y_train)

y_predict=model.predict(X_test)

print('Error on test data:')
print('MSE: %.1f' % mse(y_test,y_predict))
print('RMSE: %.1f' % mse(y_test,y_predict,squared=False))
print('R2 : %.4f' %  r2_score(y_test,y_predict))

Error on test data:
MSE: 0.8
RMSE: 0.9
R2 : 0.4669


# Lasso

In [207]:
model_L = Lasso(alpha=0.00001, fit_intercept=True)

model_L.fit(X_train, y_train)
y_predict=model_L.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict))

R2 : 0.4490


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.763e+01, tolerance: 2.697e-01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


## Compare answers with original values

In [ ]:
def plotGraph(y_test,y_predict,regressorName):
    if max(y_test) >= max(y_predict):
        my_range = int(max(y_test))
    else:
        my_range = int(max(y_predict))
    plt.scatter(range(len(y_test)), y_test, color='blue')
    plt.scatter(range(len(y_predict)), y_predict, color='red')
    plt.title(regressorName)
    plt.show()
    return


plotGraph(y_test, y_predict_Elastic, "Elastic: RED - predicted. BLUE - original")
plotGraph(y_test, y_predict, "Ridge: RED - predicted. BLUE - original")

# Cross validation

In [ ]:
def cross_val(model, X_train, y_train):
  scoring = {'R2': 'r2',
            '-MSE': 'neg_mean_squared_error',
            '-MAE': 'neg_mean_absolute_error',
            'Max': 'max_error'}


  scores = cross_validate(model, X_train, y_train,
                        scoring=scoring, cv=ShuffleSplit(n_splits=5, random_state=42) )

  print('CV Results')
  DF_cv_linreg = pd.DataFrame(scores)
  display(DF_cv_linreg)
  print('\n')
  print(DF_cv_linreg.mean()[2:])
  print('\n')

cross_val(model_Elastic, X_train, y_train)

In [ ]:
model = Ridge(alpha=0.0001, max_iter=10000)

model.fit(X_train, y_train)

y_predict = model_Elastic.predict(test)

#y_predict = np.exp(y_predict)
Submission['polution'] = y_predict
Submission

In [ ]:
! rm /kaggle/working/water-pollution/Submission.csv
! mkdir /kaggle/working/water-pollution/
Submission['polution'][27] = 2.0

In [ ]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)

In [ ]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"hydramst","key": "" }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cat ~/.kaggle/kaggle.json

In [ ]:
#! kaggle competitions submit -f /kaggle/working/water-pollution/Submission.csv -m "Message" sf-matml-2022-regression